<a href="https://colab.research.google.com/github/hoax12/Alfie/blob/main/Alfie_Chunking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Sprint 1: High-Performance Data Ingestion & Retrieval
Data: 19,187 Semantic Chunks from Algebra Textbooks.

Tech: HuggingFace Embeddings (all-mpnet-base-v2), BM25 Keyword Search, and CrossEncoder Reranking.

Status: Production-ready vector foundation complete.

In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
# Install LangChain, PDF loaders, and the Semantic Chunker
!pip install -qU langchain_experimental langchain_openai langchain_community pypdf tiktoken
# 1. Install ChromaDB
!pip install -qU chromadb
#ChromaDB: This is an open-source Vector Database.
from langchain_community.vectorstores import Chroma
import os
from langchain_community.document_loaders import PyPDFLoader

from langchain_experimental.text_splitter import SemanticChunker

The Semantic Ingestion Script

Summary of the Flow:
Find a PDF.

Extract the text and page numbers.
Analyze the meaning (using your all-mpnet-base-v2 model).

Cut the text into chunks based on topic shifts.
Store those chunks in a list with their "ID card" (filename and page number).

In [ ]:
# Install HuggingFace and Sentence Transformers
import os
from langchain_community.document_loaders import PyPDFLoader
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_experimental.text_splitter import SemanticChunker

# 1. Initialize Local Embeddings (No API Key Required)
# We use 'all-mpnet-base-v2' because it's the gold standard for academic sentence similarity
print("Initializing local embedding model...")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

# 2. Setup the Semantic Chunker
# 'percentile' threshold helps capture topic shifts in structured textbooks
text_splitter = SemanticChunker(embeddings, breakpoint_threshold_type="percentile")

# 3. Path Configuration
DATA_FOLDER = "/content/drive/MyDrive/Alfie/Data"
all_chunks = []

# 4. The Ingestion Loop
print("Starting PDF Ingestion...")
for filename in os.listdir(DATA_FOLDER): #os.listdir() method is used to retrieve a list containing the names of all entries (files and directories) within a specified path.
    if filename.endswith(".pdf"):
        file_path = os.path.join(DATA_FOLDER, filename)
        print(f"--- Processing: {filename} ---")

        try:
            loader = PyPDFLoader(file_path) # This "opens" the PDF and extracts the raw text. It also captures metadata (like which page number the text came from).
            pages = loader.load()

            # We split the entire document while keeping track of page numbers
            chunks = text_splitter.split_documents(pages)

            # Append to our master list
            all_chunks.extend(chunks)
            print(f"Added {len(chunks)} chunks from {filename}")

        except Exception as e:
            print(f"Error processing {filename}: {e}")

print(f"\n✅ Success! Total semantic chunks created: {len(all_chunks)}")

# Quick Check: Look at the first chunk's metadata
if all_chunks:
    print(f"Sample Metadata: {all_chunks[0].metadata}")

STORAGE PHASE - Saving the chunks to vectorDB


In [ ]:
# 1. Install ChromaDB
!pip install -qU chromadb
#ChromaDB: This is an open-source Vector Database.
from langchain_community.vectorstores import Chroma

# 2. Define where to save the database on your Drive
PERSIST_DIRECTORY = "/content/drive/MyDrive/Alfie/VectorDB"

# 3. Create the Vector Store and Save the Chunks
print("Creating Vector Database... this might take a few minutes for 6000+ chunks.")
vector_db = Chroma.from_documents(
    documents=all_chunks,
    embedding=embeddings, # The HuggingFace model we initialized earlier
    persist_directory=PERSIST_DIRECTORY
)

# 4. Finalize the save
vector_db.persist() # Fuck it we can remove this since we are already saing it in the drive.
print(f"✅ Database saved to {PERSIST_DIRECTORY}")

Entire flow so far -

Sentence Splitting: The PDF text is broken into individual sentences.

Temporary Embedding: Each sentence is turned into a vector just to calculate where to cut.

Semantic Chunking: Using Cosine Similarity, sentences are grouped into chunks (e.g., sentences 1-5 become "Chunk A").

Final Embedding: Once "Chunk A" is created, the entire chunk is sent to the embedding model one last time to get a single "Master Vector" representing that whole paragraph.

Storage: That Master Vector + the Text + the Metadata (page number/filename) are saved into ChromaDB.

Why do we embed a second time?
We do this because a single vector representing a whole paragraph is much more powerful for search than just having a bunch of individual sentence vectors. It captures the "full context" of the idea.


In [18]:
!pip install -qU langchain-huggingface
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.3/566.3 kB 12.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 5.0.0 requires huggingface-hub<2.0,>=1.3.0, but you have huggingface-hub 0.36.1 which is incompatible.


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [19]:
# 1. Load the DB (This proves it saved correctly)
vector_db = Chroma(
    persist_directory="/content/drive/MyDrive/Alfie/VectorDB",
    embedding_function=embeddings
)

# 2. Check the count
print(f"Verified Chunks in DB: {vector_db._collection.count()}")

# 3. Test Retrieval Quality
query = "What are the rules for adding and subtracting polynomials?"
results = vector_db.similarity_search(query, k=3)

print("\n--- Top Result ---")
if results:
    print(f"Source: {results[0].metadata.get('source')}")
    print(f"Content Snippet: {results[0].page_content[:400]}...")
else:
    print("❌ No results found. Something went wrong with the index!")

/tmp/ipython-input-1654773432.py:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vector_db = Chroma(


Verified Chunks in DB: 19187

--- Top Result ---
Source: /content/drive/MyDrive/Alfie/Data/Algebra1.pdf
Content Snippet: You can use the following video if you need some help. Access multimedia content (http://openstax.org/books/algebra-1/pages/6-1-2-adding-and-subtracting-polynomials)
12. 974 6.1 • Add and Subtract Polynomials
Access for free at openstax.org...


# Task
Create a hybrid retrieval system by manually combining and deduplicating results from both BM25 and vector retrievers, then apply a reranking model to these combined documents and display the reranked results, explaining the benefits of this approach.

## Manual Combination and Deduplication

### Subtask:
Execute both the BM25 retriever and the vector retriever separately, then combine their results and remove any duplicate documents. This will provide a comprehensive list of potentially relevant documents.


**Reasoning**:
Execute both BM25 and vector retrievers separately for the given query, then combine their results and deduplicate them to fulfill the subtask.



**Reasoning**:
The previous cell failed because `bm25_retriever` and `vector_retriever` were not defined. This happened because the cell where they were initialized (`ZbIMELbkiwQ8`) failed due to an `ImportError` related to `EnsembleRetriever`. To fix this, I will re-initialize `bm25_retriever` and `vector_retriever` in the current cell, omitting the problematic `EnsembleRetriever` part as it's not needed for the current manual deduplication subtask.



In [20]:
from langchain_community.retrievers import BM25Retriever
from langchain_community.vectorstores import Chroma

query = "What is the FOIL method?"

# Re-initialize the retrievers as their previous initialization failed
# 1. Initialize the Keyword Retriever (BM25)
bm25_retriever = BM25Retriever.from_documents(all_chunks)
bm25_retriever.k = 3 # Top 3 keyword matches

# 2. Re-load Chroma and Initialize the Vector Retriever
# `embeddings` and `PERSIST_DIRECTORY` are available from previous cells.
vector_db = Chroma(
    persist_directory=PERSIST_DIRECTORY,
    embedding_function=embeddings
)
vector_retriever = vector_db.as_retriever(search_kwargs={"k": 3})


# 1. Use the existing bm25_retriever to retrieve documents
bm25_docs = bm25_retriever.invoke(query)
print(f"BM25 retrieved {len(bm25_docs)} documents.")

# 2. Use the existing vector_retriever to retrieve documents
vector_docs = vector_retriever.invoke(query)
print(f"Vector retrieved {len(vector_docs)} documents.")

# 3. Combine bm25_docs and vector_docs into a single list
combined_docs = bm25_docs + vector_docs
print(f"Combined documents before deduplication: {len(combined_docs)}")

# 4. Create a new list to store deduplicated documents
deduplicated_docs = []
seen_page_content = set()

for doc in combined_docs:
    if doc.page_content not in seen_page_content:
        deduplicated_docs.append(doc)
        seen_page_content.add(doc.page_content)

# 5. Print the number of documents before and after deduplication
print(f"Combined documents after deduplication: {len(deduplicated_docs)}")

print("\n--- Deduplicated Results ---")
for i, doc in enumerate(deduplicated_docs):
    print(f"Result {i+1} (Source: {doc.metadata.get('source', 'N/A')} - Page: {doc.metadata.get('page_label', 'N/A')}):")
    print(f"{doc.page_content[:200]}...\n")

NameError: name 'all_chunks' is not defined

## Implement Reranking

### Subtask:
Install a suitable reranking library (e.g., using a cross-encoder from `sentence-transformers`), load a reranker model, and then apply it to the combined and deduplicated documents to reorder them based on their relevance to the query.


**Reasoning**:
First, I need to install the `sentence-transformers` library to use the `CrossEncoder` for reranking.



In [ ]:
pip install -qU sentence-transformers

**Reasoning**:
Now that `sentence-transformers` is installed, I will import `CrossEncoder`, load the specified reranker model, prepare the input for reranking by pairing the query with each deduplicated document's content, apply the reranker to get relevance scores, and then sort the documents based on these scores to obtain the reranked results. I will then print the reranked results along with their sources and content snippets.



In [ ]:
from sentence_transformers import CrossEncoder

# 1. Import and Load a pre-trained cross-encoder model
# Using 'cross-encoder/ms-marco-MiniLM-L-6-v2' as suggested
print("Loading CrossEncoder model...")
reranker_model = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')
print("CrossEncoder model loaded.")

# 2. Prepare input for the reranker
# The reranker expects a list of (query, document_text) pairs
reranker_inputs = [[query, doc.page_content] for doc in deduplicated_docs]

# 3. Get relevance scores from the reranker
print("Reranking documents...")
relevance_scores = reranker_model.predict(reranker_inputs)
print("Documents reranked.")

# 4. Pair documents with their scores and sort them
doc_score_pairs = list(zip(deduplicated_docs, relevance_scores))

# Sort in descending order based on relevance score
reranked_docs_with_scores = sorted(doc_score_pairs, key=lambda x: x[1], reverse=True)

# Store the reranked documents (without scores for a clean list)
reranked_docs = [doc for doc, score in reranked_docs_with_scores]

print("\n--- Reranked Results ---")
for i, (doc, score) in enumerate(reranked_docs_with_scores):
    print(f"Result {i+1} (Score: {score:.4f}, Source: {doc.metadata.get('source', 'N/A')} - Page: {doc.metadata.get('page_label', 'N/A')}):")
    print(f"{doc.page_content[:200]}...\n")

print("\nBenefits of this approach:")
print("- **Improved Relevance**: Reranking models are trained to identify fine-grained semantic matches, often improving the quality of top results compared to initial retrieval.")
print("- **Hybrid Strength**: Combining keyword (BM25) and semantic (vector) retrieval ensures broad coverage, while reranking refines the results for precision.")
print("- **Better User Experience**: More relevant results at the top lead to a more efficient and satisfying information retrieval experience.")

## Final Task

### Subtask:
Display the reranked documents and explain the benefits of this hybrid retrieval approach.


## Summary:

### Q&A
The task asked to display the reranked documents and explain the benefits of the hybrid retrieval approach. This was successfully achieved by showing the reordered list of documents with their relevance scores and explicitly listing the advantages.

### Data Analysis Key Findings
*   Initially, both BM25 and vector retrievers were re-initialized due to a `NameError`.
*   For the query "What is the FOIL method?", the BM25 retriever retrieved 3 documents, and the vector retriever also retrieved 3 documents.
*   After combining the results from both retrievers, the total count was 6 documents.
*   Deduplication based on `page_content` reduced the combined list to 5 unique documents, indicating at least one overlap between the two retrieval methods.
*   A `CrossEncoder` model, specifically 'cross-encoder/ms-marco-MiniLM-L-6-v2', was successfully loaded and used for reranking.
*   The reranking process effectively reordered the 5 deduplicated documents based on their relevance to the query.
*   Documents highly relevant to the "FOIL method" received high relevance scores (e.g., 7.7543, 6.3138), placing them at the top.
*   Less relevant documents received significantly lower scores (e.g., 0.4131, -10.6818), indicating their lower priority.

### Insights or Next Steps
*   The hybrid retrieval system, combining BM25, vector search, and a cross-encoder reranker, provides a robust method for retrieving highly relevant documents by leveraging both keyword and semantic matching, followed by fine-grained relevance scoring.
*   Further evaluation could involve quantitative metrics (e.g., NDCG, MRR) against a gold standard to objectively measure the improvement in retrieval effectiveness provided by the reranking step.


The Full Pipeline So Far:
Step	Action	Result
1. Ingestion	Semantic Chunking	6,000 meaningful text snippets.
2. Storage	ChromaDB	Vector coordinates for all snippets.
3. Retrieval	BM25 + Vector	A "rough list" of the 6 most likely chunks.
4. Reranking	Cross-Encoder	The 1 absolute best chunk moved to the top.

🗺️ Sprint 2: Building the Knowledge Skeleton

In [21]:
import re
import pandas as pd

def robust_toc_parser(book_name, text):
    data = []
    current_unit = "General"

    # 1. Clean the noise (OpenStax footers and empty lines)
    lines = [line.strip() for line in text.split('\n') if line.strip()]
    lines = [line for line in lines if "openstax.org" not in line.lower()]

    for line in lines:
        # Pattern 1: High-level Units/Chapters (e.g., "UNIT 1...", "1 Prerequisites...", or "Whole Numbers 7")
        # Matches: "UNIT 1 LINEAR EQUATIONS", "1 Prerequisites 7", "Whole Numbers 7"
        unit_match = re.match(r"^(?:UNIT\s+)?(\d+)?\s*([a-zA-Z\s,]+)\s+(\d+)$", line)

        # Pattern 2: Lessons (e.g., "1.1 Real Numbers 7")
        lesson_match = re.match(r"^(\d+\.\d+)\s+(.*?)\s+(\d+)$", line)

        # Pattern 3: Subtopics (e.g., "1.1.1 Understanding Value 43")
        subtopic_match = re.match(r"^(\d+\.\d+\.\d+)\s+(.*?)\s+(\d+)$", line)

        if unit_match:
            # If there's a number, use it, else just the title
            u_id = unit_match.group(1) if unit_match.group(1) else "0"
            current_unit = f"Unit {u_id}: {unit_match.group(2).strip()}"

        elif lesson_match:
            data.append({
                "book": book_name,
                "level": "lesson",
                "id": lesson_match.group(1),
                "title": lesson_match.group(2).strip(),
                "page": int(lesson_match.group(3)),
                "parent_unit": current_unit
            })

        elif subtopic_match:
            data.append({
                "book": book_name,
                "level": "subtopic",
                "id": subtopic_match.group(1),
                "title": subtopic_match.group(2).strip(),
                "page": int(subtopic_match.group(3)),
                "parent_unit": current_unit
            })

    return data

In [22]:
import re
import pandas as pd

# The raw text you provided
# 1. Store ToCs in a Dictionary
tocs = {
    "Algebra_1": """ UNIT 1 LINEAR EQUATIONS
Unit 1 Overview and Readiness 35
Unit 1 Overview 35
Solve Linear Equations: Mini-Lesson Review 35
Classify Equations: Mini-Lesson Review 37
Find Coordinates: Mini-Lesson Review 39
Unit 1 Overview: Wrap Up 41
1.1 Exploring Expressions and Equations 43
1.1.0 Lesson Overview 43
1.1.1 Understanding Value 43
1.1.2 Creating Expressions to Estimate Cost, Part 1 43
1.1.3 Understanding Constraints 46
1.1.4 Creating Expressions to Estimate Cost, Part 2 48
1.1.5 Practice 49
1.1.6 Lesson Summary 51
1.2 Writing Equations to Model Relationships, Part 1 53
1.2.0 Lesson Overview 53
1.2.1 Finding the Percent of 200 53
1.2.2 Modeling with Equations to Find Edges in Platonic Solids 53
1.2.3 Writing Equations to Represent Relationships 56
1.2.4 Writing Equations to Represent Relationships with Percentages 58
1.2.5 Writing Equations to Represent T-Shirts 59
1.2.6 Practice 60
1.2.7 Lesson Summary 62
1.3 Writing Equations to Model Relationships, Part 2 63
1.3.0 Lesson Overview 63
1.3.1 Finding a Relationship between x and y 63
1.3.2 Describing Relationships Using Words and Equations 63
1.3.3 Identifying and Representing Relationships 66
1.3.4 Writing an Equation to Represent a Relationship 68
1.3.5 Practice 69
1.3.6 Lesson Summary 71
1.4 Equations and Their Solutions 73
1.4.0 Lesson Overview 73
1.4.1 Finding the Solution to an Equation in One Variable, Part 1 73
1.4.2 Writing Equations to Represent Constraints 73
1.4.3 Finding the Solution to an Equation in Two Variables 75
1.4.4 Finding the Solution to an Equation in One Variable, Part 2 76
1.4.5 Practice 78
1.4.6 Lesson Summary 80
1.5 Equations and Their Graphs 81
1.5.0 Lesson Overview 81
1.5.1 Analyzing Graphs of Linear Equations 81
1.5.2 Graphing Linear Functions in Two Variables 82
1.5.3 Examining an Equation in Two Variables and Its Graph, Part 1 83
1.5.4 Writing, Graphing, and Solving a Linear Equation 86
1.5.5 Examining an Equation in Two Variables and Its Graph, Part 2 89
1.5.6 Practice 90
1.5.7 Lesson Summary 92
1.6 Equivalent Equations 93
1.6.0 Lesson Overview 93
1.6.1 Exploring Equivalent Expressions 93
1.6.2 Expressing Relationships as Equations 93
1.6.3 Exploring Related Equations 95
1.6.4 Recognizing Related Equations 97
1.6.5 Practice 98
1.6.6 Lesson Summary 100
1.7 Explaining Steps for Rewriting Equations 101
1.7.0 Lesson Overview 101
1.7.1 Determining if Zero Is a Solution 101
1.7.2 Explaining Acceptable Moves to Solve an Equation 101
1.7.3 Understanding Equations with No Solution or Infinitely Many 104
1.7.4 Solving Equations by Dividing Values 107
1.7.5 Practice 108
1.7.6 Lesson Summary 109
1.8 Choosing the Correct Variable to Solve For, Part 1 111
1.8.0 Lesson Overview 111
1.8.1 Expressing Relationships between Two Quantities 111
1.8.2 Identifying the Most Useful Form of an Equation 112
1.8.3 Writing Equations and Isolating Variables 116
1.8.4 Writing Equations Given Dimensions of Shapes 118
1.8.5 Practice 119
1.8.6 Lesson Summary 121
1.9 Choosing the Correct Variable to Solve For, Part 2 123
1.9.0 Lesson Overview 123
Access for free at openstax.org
1.9.1 Rearranging an Equation in Three Variables 123
1.9.2 Writing an Equation to Represent a Constraint 124
1.9.3 Writing and Rearranging Equations in Two Variables 126
1.9.4 Solving for a Specified Variable 130
1.9.5 Practice 131
1.9.6 Lesson Summary 132
1.10 Connecting Equations to Graphs, Part 1 133
1.10.0 Lesson Overview 133
1.10.1 Interpreting Equations in Context 133
1.10.2 Graphing Equations in Context 134
1.10.3 Writing and Graphing Equations in Standard Form 140
1.10.4 Matching Equations and Graphs 141
1.10.5 Practice 143
1.10.6 Lesson Summary 145
1.11 Connecting Equations to Graphs, Part 2 147
1.11.0 Lesson Overview 147
1.11.1 Applying the Distributive Property to Rewrite Expressions 147
1.11.2 Relating Two-Variable Equations, Their Graphs, and Situations 147
1.11.3 Reasoning Symbolically and Abstractly about Linear Equations 151
1.11.4 Identifying the Features of a Graph 153
1.11.5 Practice 154
1.11.6 Lesson Summary 156
1.12 Writing the Equation of a Line 157
1.12.0 Lesson Overview 157
1.12.1 Using the Slope Formula 157
1.12.2 Writing an Equation Given the Slope and y -intercept 157
1.12.3 Writing an Equation Given the Slope and a Point 160
1.12.4 Writing Equations in Different Forms 162
1.12.5 Writing an Equation Given Two Points 164
1.12.6 Choosing a Method to Write an Equation 166
1.12.7 Practice 168
1.12.8 Lesson Summary 170
1.13 Lines from Tables and Graphs 171
1.13.0 Lesson Overview 171
1.13.1 Writing Linear Equations 171
1.13.2 Creating Tables from Verbal Descriptions 171
1.13.3 Matching Tables, Equations, and Graphs 174
1.13.4 Creating a Table from an Equation 176
1.13.5 Practice 177
1.13.6 Lesson Summary 179
1.14 Writing Equations of Parallel and Perpendicular Lines 181
1.14.0 Lesson Overview 181
1.14.1 Using Point-Slope Form to Write the Equation of a Line 181
1.14.2 Writing Equations for Point-Slope and Slope-Intercept 182
1.14.3 Writing an Equation of a Line Parallel to a Given Line 183
1.14.4 Writing an Equation of a Line Perpendicular to a Given Line 186
1.14.5 Writing an Equation of a Line Parallel or Perpendicular to an Axis 188
1.14.6 Drawing Parallel and Perpendicular Lines 192
1.14.7 Practice 193
1.14.8 Lesson Summary 195
1.15 Direct Variation 197
1.15.0 Lesson Overview 197
1.15.1 Solving Proportions 197
1.15.2 Modeling Equations Using Direct Variation 197
1.15.3 Using Direct Variation to Solve Application Problems, Part 1 199
1.15.4 Using Direct Variation to Solve Application Problems, Part 2 201
1.15.5 Practice 202
1.15.6 Lesson Summary 203
Project 1: Slopes and Intercepts 205
Project 1 Overview 205
Notice and Wonder Graph Features 205
Matching Graph and Equations 205
Unit 1 Wrap Up 208
UNIT 2 LINEAR INEQUALITIES AND SYSTEMS
Unit 2 Overview and Readiness 209
Unit 2 Overview 209
Determine the Ordered Pair: Mini-Lesson Review 209
Find the Slope and y-intercept: Mini-Lesson Review 212
Find the x- and y-intercepts: Mini-Lesson Review 214
Unit 2 Overview: Wrap Up 216
2.1 Writing and Graphing Systems of Equations 217
2.1.0 Lesson Overview 217
2.1.1 Understanding Constraints and Values 217
2.1.2 Writing and Graphing Equations 218
2.1.3 Meeting Constraints 222
2.1.4 Exploring a System of Equations 224
2.1.5 Practice 226
2.1.6 Lesson Summary 228
2.2 Writing Systems of Equations 229
2.2.0 Lesson Overview 229
2.2.1 Interpreting Graphs of Systems of Equations 229
2.2.2 Writing Systems of Equations from Tables 230
2.2.3 Graphs of Systems of Equations 232
2.2.4 Systems of Equations in the Real-World 237
Access for free at openstax.org
2.2.5 Practice 238
2.2.6 Lesson Summary 241
2.3 Solving Systems by Substitution 243
2.3.0 Lesson Overview 243
2.3.1 Finding Connections between Graphs and Equations 243
2.3.2 Checking Solutions in Systems 244
2.3.3 Solving Equations Using Substitution 246
2.3.4 Solving More Equations 248
2.3.5 Practice 250
2.3.6 Lesson Summary 252
2.4 Solving Systems by Elimination, Part 1 253
2.4.0 Lesson Overview 253
2.4.1 Combining Two True Equations 253
2.4.2 Adding Equations 254
2.4.3 Adding and Subtracting Systems of Equations and Using Their Graphs 256
2.4.4 Determining the Best Method for Solving a System of Equations 260
2.4.5 Practice 261
2.4.6 Lesson Summary 263
2.5 Solving Systems by Elimination, Part 2 265
2.5.0 Lesson Overview 265
2.5.1 Writing and Evaluating New Equations 265
2.5.2 Adding Two Equations in a System 266
2.5.3 Solving Systems of Linear Equations in Two Variables 268
2.5.4 Solving Systems of Equations Using Real-World Examples 270
2.5.5 Practice 271
2.5.6 Lesson Summary 273
2.6 Solving Systems by Elimination, Part 3 275
2.6.0 Lesson Overview 275
2.6.1 Multiplying Equations by a Number 275
2.6.2 Writing a New System to Solve a Given System 276
2.6.3 Finding Solutions to Unordered Sets of Equivalent Systems 278
2.6.4 Building Equivalent Systems 280
2.6.5 Practice 281
2.6.6 Lesson Summary 283
2.7 Systems of Linear Equations and Their Solutions 285
2.7.0 Lesson Overview 285
2.7.1 Systems of Linear Equations with Infinitely Many Solutions 285
2.7.2 Systems of Linear Equations with No Solution 285
2.7.3 Sorting Systems of Equations Based on Number of Solutions 287
2.7.4 Writing Consistent and Inconsistent Systems of Equations 288
2.7.5 Practice 289
2.7.6 Lesson Summary 291
2.8 Representing Situations with Inequalities 293
2.8.0 Lesson Overview 293
2.8.1 Understanding Inequality Symbols 293
2.8.2 Identifying Constraints in Inequalities 293
2.8.3 Writing Inequalities to Represent Constraints 295
2.8.4 Writing More Inequalities 296
2.8.5 Practice 297
2.8.6 Lesson Summary 299
2.9 Solutions to Inequalities 301
2.9.0 Lesson Overview 301
2.9.1 Writing Solutions to Inequalities 301
2.9.2 Graphing Inequalities 301
2.9.3 Understanding the Meaning of an Inequality 304
2.9.4 Making Sense of Inequalities and Their Solutions 306
2.9.5 Comparing Equality and Inequality 306
2.9.6 Using Technology to Visualize Inequality Solutions 309
2.9.7 Identifying Inequality Solutions on a Graph 310
2.9.8 Practice 311
2.9.9 Lesson Summary 313
2.10 Writing and Solving Inequalities in One Variable 315
2.10.0 Lesson Overview 315
2.10.1 Writing an Inequality to Represent a Constraint 315
2.10.2 Using Inequalities to Solve a Problem 315
2.10.3 Different Ways of Solving an Inequality 317
2.10.4 Matching Inequalities and Solutions 319
2.10.5 Solving Inequalities 320
2.10.6 Writing Inequalities to Represent a Situation 323
2.10.7 Practice 325
2.10.8 Lesson Summary 327
2.11 Graphing Linear Inequalities in Two Variables 329
2.11.0 Lesson Overview 329
2.11.1 Using Computation and Reasoning 329
2.11.2 Finding Solutions to Inequalities on the Coordinate Plane 329
2.11.3 Sketching Solutions to Inequalities 332
2.11.4 Shading Boundary Line Regions 338
2.11.5 Practice 339
2.11.6 Lesson Summary 341
2.12 Using Linear Inequalities as Constraints 343
2.12.0 Lesson Overview 343
2.12.1 Sketching a Graph to Represent an Equation 343
2.12.2 Writing an Inequality to Represent a Constraint 344
2.12.3 Graphing Solutions and Interpreting Points 345
2.12.4 Recognizing Points Along Boundary Line 348
2.12.5 Recognizing Points for an Inequality 348
Access for free at openstax.org
2.12.6 Practice 350
2.12.7 Lesson Summary 355
2.13 Solving Problems with Inequalities in Two Variables 357
2.13.0 Lesson Overview 357
2.13.1 Graphing Inequalities with Technology 357
2.13.2 Solving Problems with Inequalities in Two Variables 358
2.13.3 Matching Representations of Inequalities 362
2.13.4 Solving Inequality Problems Using Tables 364
2.13.5 Graphing Solutions to an Inequality 370
2.13.6 Practice 372
2.13.7 Lesson Summary 375
2.14 Solutions to Systems of Linear Inequalities in Two Variables 377
2.14.0 Lesson Overview 377
2.14.1 Modeling Equations to Solve a Riddle 377
2.14.2 Finding a Pair of Values that Satisfies Multiple Inequalities 377
2.14.3 Writing Systems of Inequalities that Represent Situations 380
2.14.4 Graphing Solutions of Systems of Inequalities 383
2.14.5 Writing Inequalities for Situations and Representing on a Graph 387
2.14.6 Practice 388
2.14.7 Lesson Summary 390
2.15 Solving Problems with Systems of Linear Inequalities in Two Variables 391
2.15.0 Lesson Overview 391
2.15.1 Analyzing Graphs that Represent Linear Equations and Inequalities 391
2.15.2 Determining if Points on Boundary Lines Are Solutions to a System 392
2.15.3 Solving Problems to Satisfy Multiple Constraints Simultaneously 396
2.15.4 Identifying Graph Regions Included in a System’s Solution 399
2.15.5 Practice 400
2.15.6 Lesson Summary 403
Project 2: Modeling with Systems of Inequalities in Two Variables 405
Project 2 Overview 405
Determine a Solution to Inequalities 405
Interpret a Set of Mathematical Models 405
Perform Mathematical Modeling 406
Unit 2 Wrap Up 408
UNIT 3 TWO-VARIABLE STATISTICS
Unit 3 Overview and Readiness 409
Unit 3 Overview 409
Distinguish Patterns: Mini-Lesson Review 409
Identify Positive and Negative Trends: Mini-Lesson Review 412
Interpret Slope and y-intercept: Mini-Lesson Review 414
Unit 3 Overview: Wrap Up 416
3.1 Linear Models 419
3.1.0 Lesson Overview 419
3.1.1 Exploring Scatter Plots 419
3.1.2 Creating a Scatter Plot Using Data 420
3.1.3 Interpreting the Slope and Vertical Intercept of a Linear Model 424
3.1.4 Interpreting the Slope and Vertical Intercept of a Scatter Plot 426
3.1.5 Using an Equation for a Fit Line 431
3.1.6 Practice 433
3.1.7 Lesson Summary 437
3.2 Fitting Lines 439
3.2.0 Lesson Overview 439
3.2.1 Selecting the Best Line 439
3.2.2 Determining the Best Line That Fits the Data 440
3.2.3 Writing Linear Models without Technology 443
3.2.4 Assessing the Fit of a Linear Model 446
3.2.5 Working with Fitted Lines 451
3.2.6 Practice 452
3.2.7 Lesson Summary 454
3.3 Residuals 455
3.3.0 Lesson Overview 455
3.3.1 Subtracting an Estimated Value from an Actual Value 455
3.3.2 Plotting and Analyzing Residuals 456
3.3.3 Matching Residuals Graphs to Scatter Plots 459
3.3.4 Identifying Best Fit Line for Residuals on a Graph 463
3.3.5 Practice 465
3.3.6 Lesson Summary 468
3.4 The Correlation Coefficient 469
3.4.0 Lesson Overview 469
3.4.1 Comparing Scatter Plots with Linear and Nonlinear Trends 469
3.4.2 Recognizing Differences in Scatter Plots 470
3.4.3 Finding the Correlation Coefficient with Technology 474
3.4.4 Matching Correlation Coefficients 477
3.4.5 Using Correlation Coefficient to Determine Best Fit 486
3.4.6 Practice 487
3.4.7 Lesson Summary 491
3.5 Using the Correlation Coefficient 493
3.5.0 Lesson Overview 493
3.5.1 Using Bivariate Data in Context 493
3.5.2 Finding and Using Correlation Coefficient to Interpret the Strength of Linear Relationships 495
3.5.3 Using Correlation Coefficient to Describe Relationships between Two Variables 499
3.5.4 Interpreting the Correlation Coefficient 501
3.5.5 Practice 502
3.5.6 Lesson Summary 504
Access for free at openstax.org
3.6 Causal Relationships 505
3.6.0 Lesson Overview 505
3.6.1 Describe the Relationship between Variables 505
3.6.2 Describing How Two Variables Are Related 505
3.6.3 Using the Term Causal Relationship 509
3.6.4 Determining the Type of Relationship 511
3.6.5 Practice 512
3.6.6 Lesson Summary 513
Project 3: Two-Variable Statistics 515
Project 3 Overview 515
Estimate Lengths 515
Analyze Data 515
Unit 3 Wrap Up 516
UNIT 4 FUNCTIONS
Inquiry Project: Defining Functions 517
Inquiry Project Overview 517
Simulating Relationships 517
What is a function? 518
Different Kinds of Functions 520
Unit 4 Overview and Readiness 523
Unit 4 Overview 523
Find Slope: Mini-Lesson Review 523
Describe Graphs: Mini-Lesson Review 527
Extend a Pattern: Mini-Lesson Review 530
Unit 4 Overview: Wrap Up 531
4.1 Describing and Graphing Situations 533
4.1.0 Lesson Overview 533
4.1.1 Contrasting Two Relationships by Reasoning 533
4.1.2 Reasoning Graphically about the Relationship between the Two Quantities 536
4.1.3 Examining Relations and Functions 538
4.1.4 Describing Functional Relationships 541
4.1.5 Modeling Relationships between Two Variables 543
4.1.6 Practice 544
4.1.7 Lesson Summary 546
4.2 Function Notation 547
4.2.0 Lesson Overview 547
4.2.1 Interpreting Graphs to Answer Questions 547
4.2.2 Interpreting Statements Written in Function Notation 549
4.2.3 Finding a Unique Output for Each Input 551
4.2.4 Understanding Function Notation Using Application Problems 554
4.2.5 Practice 555
4.2.6 Lesson Summary 557
4.3 Interpreting Using Function Notation 559
4.3.0 Lesson Overview 559
4.3.1 Comparing Function Values 559
4.3.2 Writing Statements from Function Notation 560
4.3.3 Comparing Statements in Function Notation 562
4.3.4 Using Function Notation with a Real-World Example 563
4.3.5 Practice 565
4.3.6 Lesson Summary 567
4.4 Using Function Notation to Describe Rules, Part 1 569
4.4.0 Lesson Overview 569
4.4.1 Using Function Notation in Tables of Values 569
4.4.2 Defining Functions by a Rule 570
4.4.3 Using Tables to Find Function Rules 572
4.4.4 Writing a Rule Using Function Notation 575
4.4.5 Practice 576
4.4.6 Lesson Summary 578
4.5 Using Function Notation to Describe Rules, Part 2 579
4.5.0 Lesson Overview 579
4.5.1 Finding Solutions of Equations 579
4.5.2 Using the Vertical Line Test 579
4.5.3 Comparing Functions 583
4.5.4 Using Function Notation and Graphing Technology 584
4.5.5 Using Graphical and Algebraic Approaches for Finding Unknowns of Linear Functions 586
4.5.6 Practice 587
4.5.7 Lesson Summary 589
4.6 Features of Graphs 591
4.6.0 Lesson Overview 591
4.6.1 Interpreting Statements in Function Notation 591
4.6.2 Analyzing Graphs of Functions 592
4.6.3 Connecting Graphical and Verbal Representations of a Function 594
4.6.4 Key Features of Linear Functions 597
4.6.5 Interpreting Graphs 600
4.6.6 Practice 601
4.6.7 Lesson Summary 604
4.7 Finding Slope 605
4.7.0 Lesson Overview 605
4.7.1 Evaluating Fractions 605
4.7.2 Finding Slope From Tables, Graphs, and Points 605
4.7.3 Writing Linear Equations 610
4.7.4 Writing Equations From Two Points 612
Access for free at openstax.org
4.7.5 Practice 614
4.7.6 Lesson Summary 615
4.8 Using Graphs to Find Average Rate of Change 617
4.8.0 Lesson Overview 617
4.8.1 Comparing Changes in Output Based on Input 617
4.8.2 Average Rate of Change and Slope 617
4.8.3 Finding and Interpreting Average Rates of Change 621
4.8.4 Rate of Change of Linear Functions 623
4.8.5 Interpreting Average Rates of Change with Real-World Examples 627
4.8.6 Practice 628
4.8.7 Lesson Summary 631
4.9 Interpreting and Creating Graphs 633
4.9.0 Lesson Overview 633
4.9.1 Analyzing and Comparing the Properties of Graphs 633
4.9.2 Interpreting Graphs Without Units 635
4.9.3 Sketching Graphs of Functions 640
4.9.4 Representing Quantities in a Situation 642
4.9.5 Using Verbal Descriptions to Create Graphs of Functions 643
4.9.6 Practice 645
4.9.7 Lesson Summary 647
4.10 Comparing Graphs 649
4.10.0 Lesson Overview 649
4.10.1 Analyzing Graphs and Statements in Function Notation 649
4.10.2 Interpreting Graphs and Statements in Terms of a Situation 650
4.10.3 Comparing Functions Represented in Separate Graphs 652
4.10.4 Comparing Graphs and Statements that Represent Functions without a Context 656
4.10.5 Analyzing Functions Using Real-World Context 658
4.10.6 Practice 659
4.10.7 Lesson Summary 662
4.11 Graphing a Function Using Transformations 663
4.11.0 Lesson Overview 663
4.11.1 Graphing Linear Functions 663
4.11.2 Vertical Shifts 664
4.11.3 Horizontal Shifts 665
4.11.4 Vertical Stretches and Compressions 670
4.11.5 Horizontal Stretches and Compressions 673
4.11.6 Graphing Using Transformations 680
4.11.7 Practice 681
4.11.8 Lesson Summary 684
4.12 Domain and Range, Part 1 685
4.12.0 Lesson Overview 685
4.12.1 Determining Reasonable Inputs and Outputs 685
4.12.2 Domain: The Input of a Function 686
4.12.3 Range: The Output of a Function 688
4.12.4 Using Graphs to Determine the Domain of a Function 692
4.12.5 Using Functions to Answer Real-World Questions 692
4.12.6 Practice 693
4.12.7 Lesson Summary 695
4.13 Domain and Range, Part 2 697
4.13.0 Lesson Overview 697
4.13.1 Different Features of a Graph 697
4.13.2 Graphs of Functions 698
4.13.3 Finding Domain and Range Using a Graph 702
4.13.4 Real-World Domain and Range 705
4.13.5 Graph of a Function and the Domain and Range Connection 708
4.13.6 Practice 709
4.13.7 Lesson Summary 712
4.14 Sequences 713
4.14.0 Lesson Overview 713
4.14.1 Describe the Pattern 713
4.14.2 What Is a Sequence? 713
4.14.3 The Connection between Sequences and Terms 714
4.14.4 Rules for a Sequence 716
4.14.5 Practice 717
4.14.6 Lesson Summary 718
4.15 Introducing Geometric Sequences 719
4.15.0 Lesson Overview 719
4.15.1 What Are the Patterns? 719
4.15.2 Using Tables and Graphs to Represent Geometric Sequences 719
4.15.3 Complete the Sequence 721
4.15.4 Connecting the Common Ratio and the Term in a Sequence 723
4.15.5 Practice 724
4.15.6 Lesson Summary 726
4.16 Introducing Arithmetic Sequences 727
4.16.0 Lesson Overview 727
4.16.1 Interpreting Function Notation 727
4.16.2 What Is an Arithmetic Sequence? 727
4.16.3 A Sequence Is a Type of Function 729
4.16.4 Identify and Generate Sequences 730
4.16.5 Practice 731
4.16.6 Lesson Summary 733
4.17 Representing Sequences 735
4.17.0 Lesson Overview 735
4.17.1 Define a Sequence 735
4.17.2 What Is a Recursive Definition? 735
4.17.3 Ways to Represent a Sequence 739
Access for free at openstax.org
4.17.4 Geometric Recursive Definitions 741
4.17.5 Practice 742
4.17.6 Lesson Summary 744
4.18 The nᵗʰ Term of a Sequence 745
4.18.0 Lesson Overview 745
4.18.1 Repeated Operations 745
4.18.2 Identifying Domain for a Function 745
4.18.3 Define an Arithmetic Sequence by the nth Term 746
4.18.4 Define a Geometric Sequence by the nth Term 748
4.18.5 Interchanging between Formulas 750
4.18.6 Different Types of Equations 754
4.18.7 Practice 755
4.18.8 Lesson Summary 757
Project 4: Using Functions to Model Battery Power 759
Project 4 Overview 759
Estimate Time to Charge a Battery 759
Analyze Percent Charge Over Time 759
Write an Equation to Model Data 760
Unit 4 Wrap Up 762
UNIT 5 INTRODUCTION TO EXPONENTIAL FUNCTIONS
Unit 5 Overview and Readiness 763
Unit 5 Overview 763
Write Linear Functions from a Graph: Mini-Lesson Review 764
Interpret Linear Functions: Mini-Lesson Review 766
Evaluate Exponential Expressions: Mini-Lesson Review 767
Unit 5 Overview: Wrap Up 769
5.1 Properties of Exponents 771
5.1.0 Lesson Overview 771
5.1.1 Understanding Exponents 771
5.1.2 Using Product and Quotient Properties for Exponents 771
5.1.3 Using Zero Exponent Property and Negative Exponents 775
5.1.4 Using Power Properties for Exponents 777
5.1.5 Simplifying Exponents 781
5.1.6 Practice 783
5.1.7 Lesson Summary 785
5.2 Rational Exponents 787
5.2.0 Lesson Overview 787
5.2.1 Perfect Squares Revisited 787
5.2.2 The nth Root 787
5.2.3 Radicals and Rational Exponents 789
5.2.4 Properties of Exponents with Rationals 791
5.2.5 Practice 792
5.2.6 Lesson Summary 794
5.3 Patterns of Growth 795
5.3.0 Lesson Overview 795
5.3.1 Creating a Pictorial Representation of Exponential Growth 796
5.3.2 Exploring Different Growth Patterns 796
5.3.3 Compare and Contrast Two Patterns 799
5.3.4 Match Expressions and Tables with Situations 803
5.3.5 Describe Mathematically Changes in Growth 805
5.3.6 Practice 806
5.3.7 Lesson Summary 809
5.4 Representing Exponential Growth 811
5.4.0 Lesson Overview 811
5.4.1 Exponent Rules 811
5.4.2 Zero Exponent Rule 811
5.4.3 Exponential Change: The Growth Factor 813
5.4.4 Graphing Exponential Expressions 816
5.4.5 Interpret the Growth Factor of the Equation 820
5.4.6 Practice 821
5.4.7 Lesson Summary 824
5.5 Representing Exponential Decay 825
5.5.0 Lesson Overview 825
5.5.1 Identifying Linear and Exponential Quantities 825
5.5.2 Exponential Decay 825
5.5.3 Using Graphs to Represent Exponential Decay 827
5.5.4 Justifying Exponential Models 832
5.5.5 Writing Equations to Represent Exponential Decay 834
5.5.6 Practice 835
5.5.7 Lesson Summary 838
5.6 Negative Exponents and Scientific Notation 839
5.6.0 Lesson Overview 839
5.6.1 Checking Exponent Rules 839
5.6.2 Interpreting Negative Exponents in Exponential Growth 840
5.6.3 Interpreting Negative Exponents in Exponential Decay 842
5.6.4 Using Scientific Notation 846
5.6.5 Applying Scientific Notation 849
5.6.6 Interpreting Negative Exponents and Scientific Notation 852
5.6.7 Practice 853
5.6.8 Lesson Summary 855
5.7 Analyzing Graphs 857
5.7.0 Lesson Overview 857
5.7.1 Patterns in Real Numbers 857
5.7.2 Comparing Graphs 857
Access for free at openstax.org
5.7.3 Describing Graphs 861
5.7.4 Quantities that Change Exponentially 866
5.7.5 Practice 867
5.7.6 Lesson Summary 870
5.8 Exponential Situations as Functions 871
5.8.0 Lesson Overview 871
5.8.1 Meaning of a Function Presented Graphically 871
5.8.2 Represent a Function as a Table of Values, Graph, and Equation 871
5.8.3 Using Function Language and Notation 873
5.8.4 Function Notation 875
5.8.5 Practice 876
5.8.6 Lesson Summary 878
5.9 Interpreting Exponential Functions 879
5.9.0 Lesson Overview 879
5.9.1 Evaluating Expressions for Different Values of x 879
5.9.2 Analyze Underlying Relationship in a Graph of a Function 879
5.9.3 Graph Equations to Solve Problems 881
5.9.4 Deciding on Graphing Window 883
5.9.5 Interpret Graphs to Find Approximate Values 884
5.9.6 Practice 886
5.9.7 Lesson Summary 887
5.10 Looking at Rates of Change 889
5.10.0 Lesson Overview 889
5.10.1 Calculate an Average Rate of Change from Two Points 889
5.10.2 Explore Average Rates of Change in an Exponential Growth Function 890
5.10.3 Explore Average Rates of Change in an Exponential Decay Context 893
5.10.4 Average Rate of Change 895
5.10.5 Practice 896
5.10.6 Lesson Summary 898
5.11 Modeling Exponential Behavior 899
5.11.0 Lesson Overview 899
5.11.1 Changing the Graphing Window 899
5.11.2 Choosing an Appropriate Model 900
5.11.3 Modeling with Exponential Functions 903
5.11.4 Examine Exponential Decay in Context 904
5.11.5 Using Relationship Models to Answer Questions 908
5.11.6 Practice 909
5.11.7 Lesson Summary 910
5.12 Reasoning about Exponential Graphs, Part 1 911
5.12.0 Lesson Overview 911
5.12.1 Modeling Exponential Decay 911
5.12.2 Equations and Their Graphs 911
5.12.3 Graphs Representing Exponential Decay 913
5.12.4 Possible Equation for a Function on a Graph 916
5.12.5 Practice 918
5.12.6 Lesson Summary 921
5.13 Reasoning about Exponential Graphs, Part 2 923
5.13.0 Lesson Overview 923
5.13.1 Comparing Functions 923
5.13.2 Creating Exponential Functions from Graphs 923
5.13.3 Comparing Exponential Functions 927
5.13.4 Interpreting Features of an Exponential Graph 929
5.13.5 Practice 930
5.13.6 Lesson Summary 932
5.14 Which One Changes Faster? 933
5.14.0 Lesson Overview 933
5.14.1 Distinguish Linear and Exponential Growth 933
5.14.2 Using Tables to Compare Linear and Exponential Growth Functions 934
5.14.3 Compare Linear Functions with Exponential Functions 937
5.14.4 Using Function Notation to Compare Linear and Exponential Functions 941
5.14.5 Practice 942
5.14.6 Lesson Summary 944
5.15 Changes Over Equal Intervals 945
5.15.0 Lesson Overview 945
5.15.1 Writing Equivalent Expressions 945
5.15.2 Linear Function Constant Rate of Change 945
5.15.3 Growth Rate of an Exponential Function 947
5.15.4 Comparing Linear and Exponential Growth 950
5.15.5 Practice 952
5.15.6 Lesson Summary 954
Project 5: Introduction to Exponential Functions 955
Project 5 Overview 955
Analyze Data 955
Write Equations to Model Populations 955
Open-ended Modeling with Data Investigation 956
Unit 5 Wrap Up 958
UNIT 6 WORKING WITH POLYNOMIALS
Unit 6 Overview and Readiness 959
Unit 6 Overview 959
Simplify Expressions Using Properties of Exponents: Mini-Lesson Review 959
Simplify Expressions Using the Distributive Property: Mini-Lesson Review 962
Find the Greatest Common Factor: Mini-Lesson Review 963
Unit 6 Overview: Wrap Up 965
Access for free at openstax.org
Unit 6 Inquiry Project: Area Model Multiplication 967
Inquiry Project Overview 967
Discovering an Area Model 967
Understanding an Area Model 968
Using an Area Model 969
Applying an Area Model to Polynomials 970
6.1 Add and Subtract Polynomials 973
6.1.0 Lesson Overview 973
6.1.1 Understanding Polynomials 973
6.1.2 Adding and Subtracting Polynomials 974
6.1.3 Evaluating a Polynomial Function for a Given Value 977
6.1.4 Adding and Subtracting Polynomial Functions 979
6.1.5 Reviewing Polynomial Addition and Subtraction 982
6.1.6 Practice 983
6.1.7 Lesson Summary 984
6.2 Multiplying Polynomials 985
6.2.0 Lesson Overview 985
6.2.1 Multiplying Monomials 985
6.2.2 Multiplying Binomials 986
6.2.3 Multiplying a Polynomial by a Polynomial 991
6.2.4 Multiplying Special Products 993
6.2.5 Multiplying Polynomial Functions 996
6.2.6 Practice 997
6.2.7 Lesson Summary 999
6.3 Dividing Polynomials 1001
6.3.0 Lesson Overview 1001
6.3.1 Dividing by Monomials 1001
6.3.2 Dividing Polynomials Using Long Division 1002
6.3.3 Dividing Polynomials Using Synthetic Division 1007
6.3.4 Dividing Polynomial Functions and the Remainder Theorem 1010
6.3.5 Using the Factor Theorem 1013
6.3.6 Practice 1015
6.3.7 Lesson Summary 1017
6.4 Greatest Common Factor and Factor by Grouping 1019
6.4.0 Lesson Overview 1019
6.4.1 Finding Factors 1019
6.4.2 Finding the GCF of Two or More Expressions 1019
6.4.3 Factoring the GCF from Polynomials 1021
6.4.4 Factoring Polynomials by Grouping 1024
6.4.5 Can You Factor the Polynomial? 1026
6.4.6 Practice 1027
6.4.7 Lesson Summary 1028
6.5 Factor Trinomials 1029
6.5.0 Lesson Overview 1029
6.5.1 Undoing FOIL 1029
6.5.2 Factoring Trinomials with Leading Coefficients of 1 1030
6.5.3 Factoring Trinomials Using Trial and Error 1035
6.5.4 Factoring Trinomials Using the “ac” Method and Substitution 1039
6.5.5 Choosing Your Own Method to Factor Trinomials 1042
6.5.6 Practice 1043
6.5.7 Lesson Summary 1045
6.6 Factor Special Products 1047
6.6.0 Lesson Overview 1047
6.6.1 Identifying Perfect Square Trinomials 1047
6.6.2 Factoring Perfect Square Trinomials 1047
6.6.3 Factoring the Difference of Squares 1052
6.6.4 Sorting Polynomials by Factoring Methods 1055
6.6.5 Practice 1057
6.6.6 Lesson Summary 1059
6.7 General Strategy for Factoring Polynomials 1061
6.7.0 Lesson Overview 1061
6.7.1 Factoring Strategies for Polynomials 1061
6.7.2 Reviewing General Strategies to Factor a Polynomial 1061
6.7.3 Implementing General Strategies for Factoring 1064
6.7.4 Is the Polynomial Completely Factored? 1067
6.7.5 Practice 1068
6.7.6 Lesson Summary 1070
Project 6: Polynomials and Rectangles 1071
Project 6 Overview 1071
Areas of Rectangles 1071
Dimensions and Areas of Rectangles 1072
Create Your Own Rectangles 1072
Unit 6 Wrap Up 1074
UNIT 7 INTRODUCTION TO QUADRATIC FUNCTIONS
Unit 7 Overview and Readiness 1075
Unit 7 Overview 1075
Multiply Binomials: Mini-Lesson Review 1075
Factor Trinomials: Mini-Lesson Review 1077
Compare Linear and Exponential Functions: Mini-Lesson Review 1079
Unit 7 Overview: Wrap Up 1083
7.1 Patterns of Change 1085
7.1.0 Lesson Overview 1085
7.1.1 Noticing New Patterns of Change 1086
Access for free at openstax.org
7.1.2 The Relationship between Length and Area 1087
7.1.3 Plotting Measurements 1088
7.1.4 The Input and Output of a Function 1090
7.1.5 Practice 1091
7.1.6 Lesson Summary 1093
7.2 Introduction to Quadratic Relationships 1095
7.2.0 Lesson Overview 1095
7.2.1 Squares in a Geometric Pattern 1095
7.2.2 Constant and Exponential Change 1096
7.2.3 Quadratic Expressions 1099
7.2.4 Quadratic Relationships 1101
7.2.5 Practice 1103
7.2.6 Lesson Summary 1105
7.3 Determining if a Function is Quadratic 1107
7.3.0 Lesson Overview 1107
7.3.1 Quadratic Expressions and Area 1107
7.3.2 Writing Equations for Patterns with Squares 1108
7.3.3 Quadratic Sequences 1110
7.3.4 Is the Pattern a Quadratic Function? 1112
7.3.5 Practice 1113
7.3.6 Lesson Summary 1115
7.4 Comparing Quadratic and Exponential Functions 1117
7.4.0 Lesson Overview 1117
7.4.1 Comparing Expressions 1117
7.4.2 Exploring Linear, Exponential, and Quadratic Growth 1117
7.4.3 Comparing Exponential and Quadratic Functions 1120
7.4.4 Comparing Exponential and Quadratic Expressions 1122
7.4.5 Practice 1123
7.4.6 Lesson Summary 1125
7.5 Building Quadratic Functions to Describe Situations, Part 1 1127
7.5.0 Lesson Overview 1127
7.5.1 Numerical Patterns 1127
7.5.2 A Time-Distance Quadratic Model 1127
7.5.3 Distance as a Quadratic Function of Elapsed Time 1129
7.5.4 A Quadratic Expression that Represents the Relationship between Distance and Time 1131
7.5.5 Practice 1132
7.5.6 Lesson Summary 1134
7.6 Building Quadratic Functions to Describe Situations, Part 2 1135
7.6.0 Lesson Overview 1135
7.6.1 Using Linear Functions to Describe Constant Speed 1135
7.6.2 The Force of Gravity Change in Quadratic Functions 1136
7.6.3 Using Quadratic Functions to Describe Height 1137
7.6.4 Interpreting Graphs of Quadratic Functions 1139
7.6.5 Practice 1140
7.6.6 Lesson Summary 1142
7.7 Domain, Range, Vertex, and Zeros of Quadratic Functions 1143
7.7.0 Lesson Overview 1143
7.7.1 Comparing Graphs of Functions 1144
7.7.2 Modeling Real-World Data with Quadratic Functions 1144
7.7.3 The Domain, Vertex, and Zero of Quadratic Functions 1147
7.7.4 Relate the Domain of a Function to Its Graph 1151
7.7.5 Practice 1152
7.7.6 Lesson Summary 1155
7.8 Equivalent Quadratic Expressions 1157
7.8.0 Lesson Overview 1157
7.8.1 Area Diagrams 1157
7.8.2 Using the Distributive Property to Write Equivalent Expressions 1157
7.8.3 Using Diagrams to Find Equivalent Quadratic Expressions 1159
7.8.4 Writing Equivalent Expressions 1161
7.8.5 Practice 1163
7.8.6 Lesson Summary 1165
7.9 Standard Form and Factored Form 1167
7.9.0 Lesson Overview 1167
7.9.1 Solving Equations Using Opposite Operations 1167
7.9.2 Finding Products of Differences 1167
7.9.3 Standard and Factored Forms of Quadratic Expressions 1169
7.9.4 Expressions in Standard Form 1170
7.9.5 Practice 1171
7.9.6 Lesson Summary 1172
7.10 Graphs of Functions in Standard and Factored Forms 1173
7.10.0 Lesson Overview 1173
7.10.1 Graphs of Linear Equations 1173
7.10.2 Quadratic Forms and Their Graphs 1174
7.10.3 The x- and y-intercepts of Quadratic Expressions 1176
7.10.4 Using the Standard Form of an Equation to Find the x- and y-intercepts 1180
7.10.5 Practice 1181
7.10.6 Lesson Summary 1182
7.11 Graphing from the Factored Form 1183
7.11.0 Lesson Overview 1183
7.11.1 Finding Coordinates 1183
7.11.2 The Factored Form of a Quadratic Expression and the x-intercepts of Its Graph 1184
7.11.3 Sketching a Graph of a Quadratic Function Using at Least Three Identifiable Points 1185
7.11.4 Using the Vertex and Axis of Symmetry of Quadratics 1187
7.11.5 Graphs of Quadratic Functions 1189
7.11.6 Practice 1190
7.11.7 Lesson Summary 1192
Access for free at openstax.org
7.12 Graphing the Standard Form, Part 1 1193
7.12.0 Lesson Overview 1193
7.12.1 Using Coefficients and Constant Terms to Identify Graphs of Linear Equations 1193
7.12.2 Transformations with Quadratic Functions 1194
7.12.3 Understanding the Behaviors of a Graph in Relation to Its Quadratic Expression 1198
7.12.4 Representations of Quadratic Functions 1205
7.12.5 Identifying Graphs of Quadratic Equations 1207
7.12.6 Practice 1208
7.12.7 Lesson Summary 1212
7.13 Graphing the Standard Form, Part 2 1213
7.13.0 Lesson Overview 1213
7.13.1 Equivalent Expressions 1213
7.13.2 The Linear Term in a Quadratic Expression 1214
7.13.3 Writing Equations that Represent a Graph 1216
7.13.4 Writing Quadratic Equations from Real Solutions 1218
7.13.5 Graphing Quadratic Equations 1219
7.13.6 Practice 1221
7.13.7 Lesson Summary 1225
7.14 Graphs That Represent Situations 1227
7.14.0 Lesson Overview 1227
7.14.1 Evaluating Quadratic Functions in the Real World 1227
7.14.2 Interpreting a Functional Relationship between Two Quantities 1228
7.14.3 Analyzing Functions Using Different Representations 1230
7.14.4 Using Quadratic Functions to Solve Problems 1231
7.14.5 Identifying Key Features of Graphed Functions 1232
7.14.6 Practice 1233
7.14.7 Lesson Summary 1235
7.15 Vertex Form 1237
7.15.0 Lesson Overview 1237
7.15.1 Analyzing Two Sets of Equations 1237
7.15.2 The Vertex Form 1237
7.15.3 The Coordinates of the Vertex 1240
7.15.4 The Graph of a Function in Vertex Form 1242
7.15.5 Practice 1243
7.15.6 Lesson Summary 1245
7.16 Graphing from the Vertex Form 1247
7.16.0 Lesson Overview 1247
7.16.1 Three Forms of Quadratic Expressions 1247
7.16.2 Graph Functions in Vertex Form 1247
7.16.3 Quadratic Equations and Graphs 1250
7.16.4 Sketching A Graph of an Equation 1254
7.16.5 Practice 1255
7.16.6 Lesson Summary 1257
7.17 Changing the Vertex 1259
7.17.0 Lesson Overview 1259
7.17.1 Quadratic Expressions in Different Forms 1259
7.17.2 Modify Expressions to Translate Graphs 1260
7.17.3 Changing the Parameters of a Quadratic Expression 1264
7.17.4 Transforming the Parent Function 1266
7.17.5 Equations of Graph Translations 1267
7.17.6 Practice 1268
7.17.7 Lesson Summary 1271
Project 7: Design a Fountain 1273
Project 7 Overview 1273
Introduction to Fountain Design 1273
Design a Fountain 1273
Unit 7 Wrap Up 1276
UNIT 8 QUADRATIC EQUATIONS
Unit 8 Overview and Readiness 1277
Unit 8 Overview 1277
Write Equivalent Expressions: Mini-Lesson Review 1277
Find Intercepts: Mini-Lesson Review 1278
Match Quadratic Graphs and Their Equations: Mini-Lesson Review 1280
Unit 8 Overview: Wrap Up 1282
8.1 Finding Unknown Inputs 1283
8.1.0 Lesson Overview 1283
8.1.1 Understanding a Situation with Quadratic Equations 1283
8.1.2 Modeling a Quadratic Problem 1284
8.1.3 Formulating a Quadratic Equation to Represent the Model 1285
8.1.4 Interpreting a Solution 1288
8.1.5 Practice 1289
8.1.6 Lesson Summary 1291
8.2 When and Why Do We Write Quadratic Equations? 1293
8.2.0 Lesson Overview 1293
8.2.1 Interpreting Equations in Context 1293
8.2.2 Exploring Non-Graphing Strategies to Solve Quadratic Equations 1294
8.2.3 Solving a Quadratic Equation Set Equal to Zero 1295
8.2.4 Solving a Real-World Problem Using Quadratic Equations 1297
8.2.5 Practice 1298
8.2.6 Lesson Summary 1300
8.3 Solving Quadratic Equations by Reasoning 1301
8.3.0 Lesson Overview 1301
8.3.1 Determining the Number of Solutions 1301
8.3.2 Recognizing Pairs of Solutions 1301
Access for free at openstax.org
8.3.3 Solving More Complex Quadratic Equations 1304
8.3.4 Finding the Solutions 1306
8.3.5 Practice 1307
8.3.6 Lesson Summary 1309
8.4 Solving Quadratic Equations with the Zero Product Property 1311
8.4.0 Lesson Overview 1311
8.4.1 Introducing the Zero Product Property 1311
8.4.2 Solving Equations of Increasing Complexity Using Reasoning 1311
8.4.3 Applying the Zero Product Property to Solve a Real-World Projectile Problem 1313
8.4.4 Solving Equations Using the Zero Product Property and Reasoning 1315
8.4.5 Practice 1316
8.4.6 Lesson Summary 1318
8.5 How Many Solutions? 1319
8.5.0 Lesson Overview 1319
8.5.1 Reviewing Zero Product Property 1319
8.5.2 Interpreting Graphs to Solve Quadratic Equations 1319
8.5.3 Choosing an Effective Strategy to Solve Quadratic Equations 1323
8.5.4 Analyzing Errors When Solving Quadratic Equations 1325
8.5.5 Practice 1326
8.5.6 Lesson Summary 1328
8.6 Rewriting Quadratic Expressions in Factored Form, Part 1 1329
8.6.0 Lesson Overview 1329
8.6.1 Finding and Reasoning Unknown Factors 1329
8.6.2 Using Diagrams to Understand Equivalent Expressions 1330
8.6.3 Rewriting Quadratic Expressions in Standard Form 1334
8.6.4 Finding the Missing Numbers in Pairs of Equivalent Expressions 1336
8.6.5 Practice 1337
8.6.6 Lesson Summary 1339
8.7 Rewriting Quadratic Expressions in Factored Form, Part 2 1341
8.7.0 Lesson Overview 1341
8.7.1 Understanding Sums and Products of Integers 1341
8.7.2 Interpreting Negative Constant Terms When Factoring Quadratic Expressions 1341
8.7.3 Analyzing Factors of 100 and –100 1344
8.7.4 Finding the Missing Terms 1348
8.7.5 Practice 1349
8.7.6 Lesson Summary 1351
8.8 Rewriting Quadratic Expressions in Factored Form, Part 3 1353
8.8.0 Lesson Overview 1353
8.8.1 Evaluating Expressions Using Mental Math 1353
8.8.2 Recognizing the Expanded Product of the Difference of Two Squares 1353
8.8.3 Factoring Quadratic Equations without a Linear Term 1356
8.8.4 Determining if an Expression Can Be Rewritten in Factored Form 1358
8.8.5 Practice 1359
8.8.6 Lesson Summary 1361
8.9 Solving Quadratic Equations by Using Factored Form 1363
8.9.0 Lesson Overview 1363
8.9.1 Finding a Solution through Substitution 1363
8.9.2 Using Factored Form and the Zero Product Property to Solve Quadratic Equations 1363
8.9.3 Writing an Equation to Represent a Quadratic Function with Only One Solution 1365
8.9.4 Solving More Quadratic Equations 1367
8.9.5 Practice 1368
8.9.6 Lesson Summary 1370
8.10 Rewriting Quadratic Expressions in Factored Form, Part 4 1371
8.10.0 Lesson Overview 1371
8.10.1 Analyzing Various Quadratic Expressions 1371
8.10.2 Working with Quadratic Factored Form with Leading Coefficient Other Than One 1371
8.10.3 Using Technology to Find Rational Factors 1375
8.10.4 Finding the Factors of Quadratic Expressions in Standard Form 1377
8.10.5 Solving Quadratic Equations by Any Method 1380
8.10.6 Practice 1381
8.10.7 Lesson Summary 1383
8.11 Writing Quadratic Equations Given Real Solutions 1385
8.11.0 Lesson Overview 1385
8.11.1 Relating a Quadratic Function and Its Zeros 1385
8.11.2 Finding a Quadratic Function from Its Zeros 1385
8.11.3 Finding a Quadratic Function from Its Zeros and a Point 1388
8.11.4 Writing Quadratics When Given Any Format 1392
8.11.5 Practice 1393
8.11.6 Lesson Summary 1395
8.12 Using Technology to Find the Quadratic Regression 1397
8.12.0 Lesson Overview 1397
8.12.1 Graphing a Quadratic Data Set Using Technology 1397
8.12.2 Finding the Curve of Best Fit 1398
8.12.3 Making Predictions Using a Quadratic Model 1401
8.12.4 Finding the Missing Data in the Set 1403
8.12.5 Practice 1405
8.12.6 Lesson Summary 1408
Project 8: Modeling Rocket Flight 1409
Project 8 Overview 1409
Relate Zeros to the Factored Form of Quadratic Equations 1409
Convert between Factored Form and Standard Form 1410
Model a Rocket Flight Using a Quadratic Equation 1410
Unit 8 Wrap Up 1412
Access for free at openstax.org
UNIT 9 MORE QUADRATIC EQUATIONS
Unit 9 Overview and Readiness 1413
Unit 9 Overview 1413
Simplifying Radicals: Mini-Lesson Review 1413
Factoring Perfect Square Trinomials: Mini-Lesson Review 1415
Transformations: Mini-Lesson Review 1417
Unit 9 Overview: Wrap Up 1418
9.1 What Are Perfect Squares? 1419
9.1.0 Lesson Overview 1419
9.1.1 Equations with Quadratic Expressions on Both Sides of the Equal Sign 1419
9.1.2 Recognizing Structure in Perfect-Square Expressions 1419
9.1.3 Solving Multi Step Quadratic Equations Using Perfect Squares 1421
9.1.4 Explain Why an Expression Is a Perfect Square 1423
9.1.5 Practice 1424
9.1.6 Lesson Summary 1426
9.2 Completing the Square, Part 1 1427
9.2.0 Lesson Overview 1427
9.2.1 Different Forms of Perfect Squares 1427
9.2.2 Standard and Factored Forms of Perfect Squares 1427
9.2.3 Completing the Square 1429
9.2.4 Making a Perfect Square 1432
9.2.5 Practice 1433
9.2.6 Lesson Summary 1436
9.3 Completing the Square, Part 2 1437
9.3.0 Lesson Overview 1437
9.3.1 Solving Equations with Fractions 1437
9.3.2 Using Completing the Square to Solve Equations 1437
9.3.3 Solve by Completing the Square 1440
9.3.4 Completing the Square with Fractions 1442
9.3.5 Practice 1443
9.3.6 Lesson Summary 1446
9.4 Completing the Square, Part 3 1447
9.4.0 Lesson Overview 1447
9.4.1 Perfect Squares with Coefficients Other than 1 1447
9.4.2 Rewrite Squared Expressions 1447
9.4.3 Standard Form and Squared Factors 1449
9.4.4 Three Different Methods for Solving an Equation 1451
9.4.5 Perfect Square and Squared Factors 1452
9.4.6 Practice 1453
9.4.7 Lesson Summary 1455
9.5 Quadratic Equations with Irrational Solutions 1457
9.5.0 Lesson Overview 1457
9.5.1 Roots of Squares 1457
9.5.2 Solutions Written as Square Roots 1457
9.5.3 Finding Irrational Solutions by Completing the Square 1459
9.5.4 Finding Exact Solutions 1461
9.5.5 Practice 1462
9.5.6 Lesson Summary 1466
9.6 The Quadratic Formula 1467
9.6.0 Lesson Overview 1467
9.6.1 Evaluating Expressions 1467
9.6.2 Choosing the Best Method to Solve an Equation 1467
9.6.3 The Quadratic Formula 1469
9.6.4 Solving Equations Using the Quadratic Formula 1471
9.6.5 Practice 1472
9.6.6 Lesson Summary 1474
9.7 Applying the Quadratic Formula 1475
9.7.0 Lesson Overview 1475
9.7.1 Evaluating Variable Expressions 1475
9.7.2 Common Calculation Errors When Using the Quadratic Formula 1475
9.7.3 Different Methods of Checking Solutions of Quadratic Equations 1478
9.7.4 Practice Spotting Calculation Errors 1480
9.7.5 Practice 1481
9.7.6 Lesson Summary 1483
9.8 Deriving the Quadratic Formula 1485
9.8.0 Lesson Overview 1485
9.8.1 Deriving the Quadratic Formula, Part 1 1486
9.8.2 Deriving the Quadratic Formula, Part 2 1488
9.8.3 Understanding That the Quadratic Formula Is the Combined Steps of Completing the Square
1490
9.8.4 Practice 1492
9.8.5 Lesson Summary 1494
9.9 Writing Quadratics in Different Forms 1495
9.9.0 Lesson Overview 1495
9.9.1 Finding the Vertex in Vertex Form 1495
9.9.2 Different Forms of Quadratics 1495
9.9.3 Vertex Form Given a Vertex and Point 1497
9.9.4 Rewriting Quadratics 1498
9.9.5 Practice 1500
9.9.6 Lesson Summary 1501
9.10 Rewriting Quadratic Expressions in Vertex Form 1503
9.10.0 Lesson Overview 1503
9.10.1 The Vertex and Intercepts of a Function 1503
9.10.2 Expanding from Factored Form to Standard Form 1504
9.10.3 Converting from Standard Form to Vertex Form 1505
9.10.4 Rewriting Expressions in Vertex Form 1506
Access for free at openstax.org
9.10.5 Different Forms of Quadratic Expressions 1507
9.10.6 Vertex Form and Coordinates of the Vertex 1508
9.10.7 Practice 1509
9.10.8 Lesson Summary 1512
9.11 Using Quadratic Expressions in Vertex Form to Solve Problems 1513
9.11.0 Lesson Overview 1513
9.11.1 Maximum and Minimum Value of a Function 1513
9.11.2 Does the Vertex Represent the Minimum or Maximum Value? 1514
9.11.3 Comparing Maximums between Quadratics 1517
9.11.4 Maximum, Minimum, and Analyzing the Vertex 1519
9.11.5 Practice 1520
9.11.6 Lesson Summary 1522
Project 9: Using Quadratic Equations to Model Situations and Solve Problems
1523
Project 9 Overview 1523
Equations of Two Lines and a Curve 1523
Analyze a Dive 1523
A Linear Function and a Quadratic Function 1523
Profit from a River Cruise 1524
Unit 9 Wrap Up 1526
Index 1527""",
    "College_Algebra": """
1 Prerequisites 7
Introduction to Prerequisites 7
1.1 Real Numbers: Algebra Essentials 7
1.2 Exponents and Scientific Notation 24
1.3 Radicals and Rational Exponents 39
1.4 Polynomials 50
1.5 Factoring Polynomials 59
1.6 Rational Expressions 68
Chapter Review 76
Exercises 79
2 Equations and Inequalities 83
Introduction to Equations and Inequalities 83
2.1 The Rectangular Coordinate Systems and Graphs 83
2.2 Linear Equations in One Variable 98
2.3 Models and Applications 115
2.4 Complex Numbers 125
2.5 Quadratic Equations 134
2.6 Other Types of Equations 149
2.7 Linear Inequalities and Absolute Value Inequalities 161
Chapter Review 172
Exercises 175
3 Functions 181
Introduction to Functions 181
3.1 Functions and Function Notation 181
3.2 Domain and Range 205
3.3 Rates of Change and Behavior of Graphs 223
3.4 Composition of Functions 239
3.5 Transformation of Functions 255
3.6 Absolute Value Functions 287
3.7 Inverse Functions 295
Chapter Review 310
Exercises 314
4 Linear Functions 323
Introduction to Linear Functions 323
4.1 Linear Functions 323
4.2 Modeling with Linear Functions 360
4.3 Fitting Linear Models to Data 374
Chapter Review 388
Exercises 389
5 Polynomial and Rational Functions 399
Introduction to Polynomial and Rational Functions 399
5.1 Quadratic Functions 400
5.2 Power Functions and Polynomial Functions 419
5.3 Graphs of Polynomial Functions 438
5.4 Dividing Polynomials 460
5.5 Zeros of Polynomial Functions 471
5.6 Rational Functions 484
5.7 Inverses and Radical Functions 508
5.8 Modeling Using Variation 521
Chapter Review 531
Exercises 535
6 Exponential and Logarithmic Functions 541
Introduction to Exponential and Logarithmic Functions 541
6.1 Exponential Functions 542
6.2 Graphs of Exponential Functions 561
6.3 Logarithmic Functions 575
6.4 Graphs of Logarithmic Functions 584
6.5 Logarithmic Properties 606
6.6 Exponential and Logarithmic Equations 618
6.7 Exponential and Logarithmic Models 631
6.8 Fitting Exponential Models to Data 650
Chapter Review 666
Exercises 671
7 Systems of Equations and Inequalities 681
Introduction to Systems of Equations and Inequalities 681
7.1 Systems of Linear Equations: Two Variables 682
7.2 Systems of Linear Equations: Three Variables 701
7.3 Systems of Nonlinear Equations and Inequalities: Two Variables 713
7.4 Partial Fractions 725
7.5 Matrices and Matrix Operations 735
7.6 Solving Systems with Gaussian Elimination 748
7.7 Solving Systems with Inverses 762
7.8 Solving Systems with Cramer's Rule 777
Chapter Review 790
Exercises 793
8 Analytic Geometry 801
Introduction to Analytic Geometry 801
8.1 The Ellipse 802
8.2 The Hyperbola 818
8.3 The Parabola 835
8.4 Rotation of Axes 851
8.5 Conic Sections in Polar Coordinates 866
Chapter Review 877
Exercises 879
9 Sequences, Probability, and Counting Theory 885
Introduction to Sequences, Probability and Counting Theory 885
Access for free at openstax.org
9.1 Sequences and Their Notations 885
9.2 Arithmetic Sequences 900
9.3 Geometric Sequences 912
9.4 Series and Their Notations 921
9.5 Counting Principles 935
9.6 Binomial Theorem 946
9.7 Probability 953
Chapter Review 964
""",
    "Prealgebra": """Whole Numbers 7
Introduction 7
1.1 Introduction to Whole Numbers 7
1.2 Add Whole Numbers 22
1.3 Subtract Whole Numbers 38
1.4 Multiply Whole Numbers 51
1.5 Divide Whole Numbers 68
Chapter Review 85
Exercises 87
2 The Language of Algebra 95
Introduction to the Language of Algebra 95
2.1 Use the Language of Algebra 95
2.2 Evaluate, Simplify, and Translate Expressions 113
2.3 Solving Equations Using the Subtraction and Addition Properties of Equality 126
2.4 Find Multiples and Factors 139
2.5 Prime Factorization and the Least Common Multiple 152
Chapter Review 162
Exercises 165
3 Integers 171
Introduction to Integers 171
3.1 Introduction to Integers 171
3.2 Add Integers 186
3.3 Subtract Integers 201
3.4 Multiply and Divide Integers 219
3.5 Solve Equations Using Integers; The Division Property of Equality 232
Chapter Review 244
Exercises 246
4 Fractions 253
Introduction to Fractions 253
4.1 Visualize Fractions 253
4.2 Multiply and Divide Fractions 274
4.3 Multiply and Divide Mixed Numbers and Complex Fractions 291
4.4 Add and Subtract Fractions with Common Denominators 302
4.5 Add and Subtract Fractions with Different Denominators 311
4.6 Add and Subtract Mixed Numbers 329
4.7 Solve Equations with Fractions 344
Chapter Review 359
Exercises 362
5 Decimals 367
Introduction to Decimals 367
5.1 Decimals 367
5.2 Decimal Operations 383
5.3 Decimals and Fractions 401
5.4 Solve Equations with Decimals 413
5.5 Averages and Probability 421
5.6 Ratios and Rate 433
5.7 Simplify and Use Square Roots 444
Chapter Review 457
Exercises 460
6 Percents 467
Introduction to Percents 467
6.1 Understand Percent 467
6.2 Solve General Applications of Percent 482
6.3 Solve Sales Tax, Commission, and Discount Applications 494
6.4 Solve Simple Interest Applications 507
6.5 Solve Proportions and their Applications 516
Chapter Review 532
Exercises 533
7 The Properties of Real Numbers 539
Introduction to the Properties of Real Numbers 539
7.1 Rational and Irrational Numbers 539
7.2 Commutative and Associative Properties 546
7.3 Distributive Property 556
7.4 Properties of Identity, Inverses, and Zero 567
7.5 Systems of Measurement 578
Chapter Review 595
Exercises 596
8 Solving Linear Equations 601
Introduction to Solving Linear Equations 601
8.1 Solve Equations Using the Subtraction and Addition Properties of Equality 601
8.2 Solve Equations Using the Division and Multiplication Properties of Equality 614
8.3 Solve Equations with Variables and Constants on Both Sides 621
8.4 Solve Equations with Fraction or Decimal Coefficients 637
Chapter Review 647
Exercises 648
9 Math Models and Geometry 651
Introduction 651
9.1 Use a Problem Solving Strategy 651
9.2 Solve Money Applications 666
9.3 Use Properties of Angles, Triangles, and the Pythagorean Theorem 679
Access for free at openstax.org
9.4 Use Properties of Rectangles, Triangles, and Trapezoids 698
9.5 Solve Geometry Applications: Circles and Irregular Figures 724
9.6 Solve Geometry Applications: Volume and Surface Area 736
9.7 Solve a Formula for a Specific Variable 755
Chapter Review 766
Exercises 769
10 Polynomials 779
Introduction to Polynomials 779
10.1 Add and Subtract Polynomials 779
10.2 Use Multiplication Properties of Exponents 788
10.3 Multiply Polynomials 801
10.4 Divide Monomials 814
10.5 Integer Exponents and Scientific Notation 830
10.6 Introduction to Factoring Polynomials 845
Chapter Review 857
Exercises 859
11 Graphs 865
Graphs 865
11.1 Use the Rectangular Coordinate System 865
11.2 Graphing Linear Equations 886
11.3 Graphing with Intercepts 904
11.4 Understand Slope of a Line 919
Chapter Review 946
""",

    "elementary algebra 2e": """1 Foundations 7
Introduction 7
1.1 Introduction to Whole Numbers 7
1.2 Use the Language of Algebra 23
1.3 Add and Subtract Integers 42
1.4 Multiply and Divide Integers 62
1.5 Visualize Fractions 77
1.6 Add and Subtract Fractions 91
1.7 Decimals 103
1.8 The Real Numbers 122
1.9 Properties of Real Numbers 136
1.10 Systems of Measurement 153
Chapter Review 170
Exercises 176
2 Solving Linear Equations and Inequalities 189
Introduction 189
2.1 Solve Equations Using the Subtraction and Addition Properties of Equality 189
2.2 Solve Equations using the Division and Multiplication Properties of Equality 203
2.3 Solve Equations with Variables and Constants on Both Sides 216
2.4 Use a General Strategy to Solve Linear Equations 226
2.5 Solve Equations with Fractions or Decimals 238
2.6 Solve a Formula for a Specific Variable 248
2.7 Solve Linear Inequalities 257
Chapter Review 273
Exercises 275
3 Math Models 281
Introduction 281
3.1 Use a Problem-Solving Strategy 281
3.2 Solve Percent Applications 299
3.3 Solve Mixture Applications 318
3.4 Solve Geometry Applications: Triangles, Rectangles, and the Pythagorean Theorem 336
3.5 Solve Uniform Motion Applications 359
3.6 Solve Applications with Linear Inequalities 372
Chapter Review 381
Exercises 383
4 Graphs 393
Introduction 393
4.1 Use the Rectangular Coordinate System 393
4.2 Graph Linear Equations in Two Variables 411
4.3 Graph with Intercepts 429
4.4 Understand Slope of a Line 442
4.5 Use the Slope-Intercept Form of an Equation of a Line 470
4.6 Find the Equation of a Line 494
4.7 Graphs of Linear Inequalities 511
Chapter Review 527
Exercises 530
5 Systems of Linear Equations 543
Introduction 543
5.1 Solve Systems of Equations by Graphing 543
5.2 Solving Systems of Equations by Substitution 563
5.3 Solve Systems of Equations by Elimination 579
5.4 Solve Applications with Systems of Equations 593
5.5 Solve Mixture Applications with Systems of Equations 613
5.6 Graphing Systems of Linear Inequalities 626
Chapter Review 640
Exercises 642
6 Polynomials 649
Introduction 649
6.1 Add and Subtract Polynomials 649
6.2 Use Multiplication Properties of Exponents 661
6.3 Multiply Polynomials 674
6.4 Special Products 688
6.5 Divide Monomials 700
6.6 Divide Polynomials 717
6.7 Integer Exponents and Scientific Notation 728
Chapter Review 747
Exercises 749
7 Factoring 757
Introduction 757
7.1 Greatest Common Factor and Factor by Grouping 757
7.2 Factor Trinomials of the Form x2+bx+c 770
7.3 Factor Trinomials of the Form ax2+bx+c 783
7.4 Factor Special Products 800
7.5 General Strategy for Factoring Polynomials 815
7.6 Quadratic Equations 825
Chapter Review 841
Exercises 843
8 Rational Expressions and Equations 847
Introduction 847
8.1 Simplify Rational Expressions 847
8.2 Multiply and Divide Rational Expressions 863
Access for free at openstax.org
8.3 Add and Subtract Rational Expressions with a Common Denominator 874
8.4 Add and Subtract Rational Expressions with Unlike Denominators 882
8.5 Simplify Complex Rational Expressions 894
8.6 Solve Rational Equations 906
8.7 Solve Proportion and Similar Figure Applications 920
8.8 Solve Uniform Motion and Work Applications 936
8.9 Use Direct and Inverse Variation 946
Chapter Review 960
Exercises 962
9 Roots and Radicals 969
Introduction 969
9.1 Simplify and Use Square Roots 969
9.2 Simplify Square Roots 978
9.3 Add and Subtract Square Roots 990
9.4 Multiply Square Roots 999
9.5 Divide Square Roots 1013
9.6 Solve Equations with Square Roots 1025
9.7 Higher Roots 1041
9.8 Rational Exponents 1057
Chapter Review 1077
Exercises 1079
10 Quadratic Equations 1085
Introduction 1085
10.1 Solve Quadratic Equations Using the Square Root Property 1085
10.2 Solve Quadratic Equations by Completing the Square 1097
10.3 Solve Quadratic Equations Using the Quadratic Formula 1112
10.4 Solve Applications Modeled by Quadratic Equations 1126
10.5 Graphing Quadratic Equations in Two Variables 1137
Chapter Review 1162""",
    "algebra and trigonometry": """1 Prerequisites 7
Introduction to Prerequisites 7
1.1 Real Numbers: Algebra Essentials 7
1.2 Exponents and Scientific Notation 24
1.3 Radicals and Rational Exponents 39
1.4 Polynomials 50
1.5 Factoring Polynomials 59
1.6 Rational Expressions 68
Chapter Review 76
Exercises 79
2 Equations and Inequalities 83
Introduction to Equations and Inequalities 83
2.1 The Rectangular Coordinate Systems and Graphs 83
2.2 Linear Equations in One Variable 98
2.3 Models and Applications 115
2.4 Complex Numbers 125
2.5 Quadratic Equations 134
2.6 Other Types of Equations 149
2.7 Linear Inequalities and Absolute Value Inequalities 161
Chapter Review 172
Exercises 175
3 Functions 181
Introduction to Functions 181
3.1 Functions and Function Notation 181
3.2 Domain and Range 205
3.3 Rates of Change and Behavior of Graphs 223
3.4 Composition of Functions 239
3.5 Transformation of Functions 255
3.6 Absolute Value Functions 287
3.7 Inverse Functions 295
Chapter Review 310
Exercises 314
4 Linear Functions 323
Introduction to Linear Functions 323
4.1 Linear Functions 323
4.2 Modeling with Linear Functions 360
4.3 Fitting Linear Models to Data 374
Chapter Review 388
Exercises 389
5 Polynomial and Rational Functions 399
Introduction to Polynomial and Rational Functions 399
5.1 Quadratic Functions 400
5.2 Power Functions and Polynomial Functions 419
5.3 Graphs of Polynomial Functions 438
5.4 Dividing Polynomials 460
5.5 Zeros of Polynomial Functions 471
5.6 Rational Functions 484
5.7 Inverses and Radical Functions 508
5.8 Modeling Using Variation 521
Chapter Review 531
Exercises 535
6 Exponential and Logarithmic Functions 541
Introduction to Exponential and Logarithmic Functions 541
6.1 Exponential Functions 542
6.2 Graphs of Exponential Functions 561
6.3 Logarithmic Functions 575
6.4 Graphs of Logarithmic Functions 584
6.5 Logarithmic Properties 606
6.6 Exponential and Logarithmic Equations 618
6.7 Exponential and Logarithmic Models 631
6.8 Fitting Exponential Models to Data 650
Chapter Review 666
Exercises 671
7 The Unit Circle: Sine and Cosine Functions 681
Introduction to The Unit Circle: Sine and Cosine Functions 681
7.1 Angles 682
7.2 Right Triangle Trigonometry 704
7.3 Unit Circle 717
7.4 The Other Trigonometric Functions 736
Chapter Review 751
Exercises 754
8 Periodic Functions 759
Introduction to Periodic Functions 759
8.1 Graphs of the Sine and Cosine Functions 759
8.2 Graphs of the Other Trigonometric Functions 779
8.3 Inverse Trigonometric Functions 800
Chapter Review 813
Exercises 814
9 Trigonometric Identities and Equations 821
Introduction to Trigonometric Identities and Equations 821
9.1 Verifying Trigonometric Identities and Using Trigonometric Identities to Simplify Trigonometric Expressions
821
9.2 Sum and Difference Identities 832
9.3 Double-Angle, Half-Angle, and Reduction Formulas 847
9.4 Sum-to-Product and Product-to-Sum Formulas 860
9.5 Solving Trigonometric Equations 868
Access for free at openstax.org
Chapter Review 884
Exercises 887
10 Further Applications of Trigonometry 893
Introduction to Further Applications of Trigonometry 893
10.1 Non-right Triangles: Law of Sines 893
10.2 Non-right Triangles: Law of Cosines 911
10.3 Polar Coordinates 925
10.4 Polar Coordinates: Graphs 939
10.5 Polar Form of Complex Numbers 958
10.6 Parametric Equations 969
10.7 Parametric Equations: Graphs 982
10.8 Vectors 994
Chapter Review 1016
Exercises 1020
11 Systems of Equations and Inequalities 1027
Introduction to Systems of Equations and Inequalities 1027
11.1 Systems of Linear Equations: Two Variables 1028
11.2 Systems of Linear Equations: Three Variables 1047
11.3 Systems of Nonlinear Equations and Inequalities: Two Variables 1059
11.4 Partial Fractions 1071
11.5 Matrices and Matrix Operations 1081
11.6 Solving Systems with Gaussian Elimination 1094
11.7 Solving Systems with Inverses 1108
11.8 Solving Systems with Cramer's Rule 1123
Chapter Review 1136
Exercises 1139
12 Analytic Geometry 1147
Introduction to Analytic Geometry 1147
12.1 The Ellipse 1148
12.2 The Hyperbola 1164
12.3 The Parabola 1181
12.4 Rotation of Axes 1197
12.5 Conic Sections in Polar Coordinates 1212
Chapter Review 1223
Exercises 1225
13 Sequences, Probability, and Counting Theory 1231
Introduction to Sequences, Probability and Counting Theory 1231
13.1 Sequences and Their Notations 1231
13.2 Arithmetic Sequences 1246
13.3 Geometric Sequences 1258
13.4 Series and Their Notations 1267
13.5 Counting Principles 1281
13.6 Binomial Theorem 1292
13.7 Probability 1299
Chapter Review 1310""",
    "intermediate algebra 2e": """
1 Foundations 7
Introduction 7
1.1 Use the Language of Algebra 7
1.2 Integers 26
1.3 Fractions 43
1.4 Decimals 57
1.5 Properties of Real Numbers 73
Chapter Review 84
Exercises 90
2 Solving Linear Equations 95
Introduction 95
2.1 Use a General Strategy to Solve Linear Equations 95
2.2 Use a Problem Solving Strategy 112
2.3 Solve a Formula for a Specific Variable 132
2.4 Solve Mixture and Uniform Motion Applications 149
2.5 Solve Linear Inequalities 168
2.6 Solve Compound Inequalities 187
2.7 Solve Absolute Value Inequalities 197
Chapter Review 209
Exercises 213
3 Graphs and Functions 223
Introduction 223
3.1 Graph Linear Equations in Two Variables 223
3.2 Slope of a Line 249
3.3 Find the Equation of a Line 273
3.4 Graph Linear Inequalities in Two Variables 288
3.5 Relations and Functions 306
3.6 Graphs of Functions 319
Chapter Review 337
Exercises 342
4 Systems of Linear Equations 355
Introduction 355
4.1 Solve Systems of Linear Equations with Two Variables 355
4.2 Solve Applications with Systems of Equations 377
4.3 Solve Mixture Applications with Systems of Equations 396
4.4 Solve Systems of Equations with Three Variables 411
4.5 Solve Systems of Equations Using Matrices 423
4.6 Solve Systems of Equations Using Determinants 436
4.7 Graphing Systems of Linear Inequalities 450
Chapter Review 466
Exercises 469
5 Polynomials and Polynomial Functions 477
Introduction 477
5.1 Add and Subtract Polynomials 477
5.2 Properties of Exponents and Scientific Notation 490
5.3 Multiply Polynomials 513
5.4 Dividing Polynomials 528
Chapter Review 541
Exercises 544
6 Factoring 551
Introduction to Factoring 551
6.1 Greatest Common Factor and Factor by Grouping 551
6.2 Factor Trinomials 559
6.3 Factor Special Products 576
6.4 General Strategy for Factoring Polynomials 588
6.5 Polynomial Equations 597
Chapter Review 613
Exercises 616
7 Rational Expressions and Functions 621
Introduction 621
7.1 Multiply and Divide Rational Expressions 621
7.2 Add and Subtract Rational Expressions 635
7.3 Simplify Complex Rational Expressions 649
7.4 Solve Rational Equations 660
7.5 Solve Applications with Rational Equations 673
7.6 Solve Rational Inequalities 698
Chapter Review 709
Exercises 711
8 Roots and Radicals 719
Introduction 719
8.1 Simplify Expressions with Roots 719
8.2 Simplify Radical Expressions 734
8.3 Simplify Rational Exponents 751
8.4 Add, Subtract, and Multiply Radical Expressions 764
8.5 Divide Radical Expressions 776
8.6 Solve Radical Equations 788
8.7 Use Radicals in Functions 802
8.8 Use the Complex Number System 810
Chapter Review 824
Exercises 827
Access for free at openstax.org
9 Quadratic Equations and Functions 835
Introduction 835
9.1 Solve Quadratic Equations Using the Square Root Property 835
9.2 Solve Quadratic Equations by Completing the Square 846
9.3 Solve Quadratic Equations Using the Quadratic Formula 860
9.4 Solve Equations in Quadratic Form 874
9.5 Solve Applications of Quadratic Equations 880
9.6 Graph Quadratic Functions Using Properties 894
9.7 Graph Quadratic Functions Using Transformations 920
9.8 Solve Quadratic Inequalities 939
Chapter Review 949
Exercises 952
10 Exponential and Logarithmic Functions 959
Introduction 959
10.1 Finding Composite and Inverse Functions 959
10.2 Evaluate and Graph Exponential Functions 975
10.3 Evaluate and Graph Logarithmic Functions 989
10.4 Use the Properties of Logarithms 1005
10.5 Solve Exponential and Logarithmic Equations 1015
Chapter Review 1026
Exercises 1029
11 Conics 1037
Introduction 1037
11.1 Distance and Midpoint Formulas; Circles 1037
11.2 Parabolas 1051
11.3 Ellipses 1070
11.4 Hyperbolas 1085
11.5 Solve Systems of Nonlinear Equations 1098
Chapter Review 1113
Exercises 1117
12 Sequences, Series and Binomial Theorem 1125
Introduction 1125
12.1 Sequences 1125
12.2 Arithmetic Sequences 1137
12.3 Geometric Sequences and Series 1147
12.4 Binomial Theorem 1161
Chapter Review 1174"""

}

# 2. Process them in a loop
master_data = []
for book, content in tocs.items():
    master_data.extend(robust_toc_parser(book, content))

master_df = pd.DataFrame(master_data)
print(f"Total structured nodes created: {len(master_df)}")
master_df.sample(5)




Total structured nodes created: 1315


,book,level,id,title,page,parent_unit
11,Algebra_1,subtopic,1.2.2,Modeling with Equations to Find Edges in Plato...,53,General
1034,College_Algebra,lesson,9.6,Binomial Theorem,946,"Unit 0: Introduction to Sequences, Probability..."
229,Algebra_1,subtopic,2.13.0,Lesson Overview,357,Unit 0: Matching Graph and Equations
658,Algebra_1,lesson,7.2,Introduction to Quadratic Relationships,1095,Unit 0: Create Your Own Rectangles
211,Algebra_1,lesson,2.11,Graphing Linear Inequalities in Two Variables,329,Unit 0: Matching Graph and Equations


In [23]:
from tqdm import tqdm

# 1. Load your existing Vector DB
vector_db = Chroma(
    persist_directory="/content/drive/MyDrive/Alfie/VectorDB",
    embedding_function=embeddings
)

# 2. Get all IDs from the database
all_data = vector_db.get()
ids = all_data['ids']
metadatas = all_data['metadatas']

updated_metadatas = []

print("Mapping 19k chunks to the Knowledge Graph...")

for meta in tqdm(metadatas):
    chunk_page = meta.get('page')
    # Get filename and clean it to match our 'book' names in master_df
    # e.g., '/path/to/Algebra_1.pdf' -> 'Algebra_1'
    chunk_source = os.path.basename(meta.get('source', '')).replace('.pdf', '')

    # Logic: Find the lesson in master_df where the page matches
    # We look for the lesson that is closest to but not greater than our current page
    relevant_book = master_df[master_df['book'] == chunk_source]

    if not relevant_book.empty:
        # Find the row where 'page' is the highest value that is still <= chunk_page
        potential_lessons = relevant_book[relevant_book['page'] <= chunk_page]
        if not potential_lessons.empty:
            best_match = potential_lessons.iloc[-1]
            meta['lesson_title'] = best_match['title']
            meta['lesson_id'] = best_match['id']
            meta['unit_title'] = best_match['parent_unit']
        else:
            meta['lesson_title'] = "Intro/Front Matter"

    updated_metadatas.append(meta)

# 3. Push the updates back in batches of 500 (prevents memory crashes)
print("\nSaving updates to Drive...")
batch_size = 500
for i in range(0, len(ids), batch_size):
    vector_db._collection.update(
        ids=ids[i:i + batch_size],
        metadatas=updated_metadatas[i:i + batch_size]
    )

print("✅ 19,187 chunks have been successfully indexed to the Knowledge Graph!")

Mapping 19k chunks to the Knowledge Graph...


100%|██████████| 19187/19187 [00:07<00:00, 2413.72it/s]



Saving updates to Drive...
✅ 19,187 chunks have been successfully indexed to the Knowledge Graph!


In [24]:
# 1. Install the visualization engine
!pip install -qU pyvis

import networkx as nx
from pyvis.network import Network
import pandas as pd

# 2. Initialize a Directed Graph
G = nx.DiGraph()

# 3. Add nodes and edges from our Master DF
# To keep the visual clean, let's visualize just one Book or one Unit first
# You can remove the filter later to see the "Mega Graph"
subset_df = master_df[master_df['book'] == 'Algebra_1']

for _, row in subset_df.iterrows():
    # Node colors based on level
    color = "#FF5733" if row['level'] == 'unit' else "#3383FF"

    G.add_node(row['id'], label=row['title'], color=color, size=20)

    if row['parent_unit'] != "General":
        G.add_edge(row['parent_unit'], row['id'])

# 4. Convert to PyVis for the Interactive UI
net = Network(height="750px", width="100%", bgcolor="#222222", font_color="white", notebook=True)
net.from_nx(G)

# Enable "Physics" so nodes bounce and organize themselves
net.toggle_physics(True)

# 5. Show the graph
net.show("algebra_mindmap.html")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 62.6 MB/s eta 0:00:00
algebra_mindmap.html


In [25]:
import IPython

# This renders the HTML file directly in a Colab output cell
IPython.display.HTML(filename='algebra_mindmap.html')